## Scrapping la asistencia de senadores en Feb~May 2018 usando datos de congresovisible.org 
Cada página tiene dos listas, una con la lista de senadores que votaron, la otra con la lista de los que se abstubieron. Como lo que queremos es determina si asistieron, ambos cuentan por igual.

In [1]:
import re
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import time
import unicodedata

In [2]:
'''Para poder comparar facil después'''
def elimina_tildes(cadena):
    s = ''.join((c for c in unicodedata.normalize('NFD',cadena) if unicodedata.category(c) != 'Mn'))
    return s

In [3]:
'''Busca las listas de senadores'''
def GetListSenators(soup):
    
    names = []

    ## Votantes
    table = soup.find('table', attrs={'id':'tabla-reporte-detallado'})
    if table != None:
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')

        for row in rows:
            name = row.find('td').get_text()
            names.append(name)
    
    ## Abstenciones
    table = soup.find('table', attrs={'id':'tabla-reporte-abstenciones'})
    if table != None:
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')

        for row in rows:
            name = row.find('td').get_text()
            names.append(name)

    ## Eliminar tildes
    names = [elimina_tildes(name) for name in names ]
    print("Found %d names" % len(names))
    return names

In [4]:
'''Busca la fecha en que se llevo a cabo la votación'''
def GetDate(soup):
    panel = soup.find_all("div", attrs={"class": "module-contenido"})

    ## Quick and dirty direct way
    # print(panel[1].contents[1].contents[7].find('p').getText()) 

    for pan in panel:
        for pa in pan.find_all('li'):
            text = pa.find('p').get_text()
            if "201" in text:
                text = text.replace('d ','')
                text = text.replace('de ','')
                print(text)
                return text

In [5]:
## Un ejemplo
page = requests.get("https://www.congresovisible.org/votaciones/%d" % 13960)
print(page.status_code)
soup = BeautifulSoup(page.content, 'html.parser')
date = GetDate(soup)
senators = GetListSenators(soup)

200
Nov 30 2016
Found 99 names


### Loop sobre posibles paginas con resultados de votaciones

In [6]:
'''Busca votaciones en las URLs con el ID en el rango recibido'''
def GetVotaciones(votaciones):
    
    for votacion in votaciones:
        print("\nVotacion %d" % votacion)
        page = requests.get("https://www.congresovisible.org/votaciones/%d" % votacion)
    
        if page.status_code == 404:
            print("Status code 404. Skipping....")
            continue
    
        soup = BeautifulSoup(page.content, 'html.parser')
        date = GetDate(soup)
        senators = GetListSenators(soup)
        df = pd.DataFrame({'senator':senators, 'date':date})
        df = df.set_index('senator')
        resultados.append(df)
    
        ## Save dataframe as csv
        df.to_csv('votaciones/votacion_%d.csv' % votacion)
    
        ## Wait 1s to avoid hammering servers too much
        time.sleep(1)

In [7]:
%%time

## Rango de URLs que posiblemente tengan resultados de votaciones
#votaciones = np.arange(13800,14000,1)
rango_votaciones = np.arange(13953,14000,1)

## Run it
#GetVotaciones(rango_votaciones)

CPU times: user 31 µs, sys: 16 µs, total: 47 µs
Wall time: 57.2 µs


#### Cada votación debe estar guardada en el folder `votaciones`  como un archivo csv con la fecha de la votación y la lista de senadores